In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt 

In [2]:
df = pd.DataFrame(pd.read_excel('VaR_practice.xlsx'))
df

,date,SREN,UBS,NOVN
0,2021-11-12,77.618309,15.735238,63.796165
1,2021-11-15,77.287575,15.735238,63.779427
2,2021-11-16,77.304970,15.848408,64.030525
3,2021-11-17,77.252754,15.975722,63.796165
4,2021-11-18,76.904610,15.697516,63.402779
...,...,...,...,...
586,2024-03-08,109.000000,27.240000,88.110001
587,2024-03-11,109.849998,27.110001,88.800003
588,2024-03-12,110.500000,27.700001,87.589996
589,2024-03-13,111.800003,27.830000,87.129997


In [3]:
#Convert to a csv file
df.to_csv("VaR_SMI.csv", index = None, header=True)
df = pd.DataFrame(pd.read_csv("VaR_SMI.csv"))
df

,date,SREN,UBS,NOVN
0,2021-11-12,77.618309,15.735238,63.796165
1,2021-11-15,77.287575,15.735238,63.779427
2,2021-11-16,77.304970,15.848408,64.030525
3,2021-11-17,77.252754,15.975722,63.796165
4,2021-11-18,76.904610,15.697516,63.402779
...,...,...,...,...
586,2024-03-08,109.000000,27.240000,88.110001
587,2024-03-11,109.849998,27.110001,88.800003
588,2024-03-12,110.500000,27.700001,87.589996
589,2024-03-13,111.800003,27.830000,87.129997


In [4]:
#Returns - we create a function and goes through a for loop to create three columns of returns
def calculate_returns(df):
    returns_df = df.pct_change().dropna()
    returns_df.columns = [col + "_return" for col in df.columns]
    return pd.concat([df, returns_df], axis=1)
data = {
    'SREN': df['SREN'],
    'UBS': df['UBS'],
    'NOVN': df['NOVN']
}
df = pd.DataFrame(data)
df_ver2 = calculate_returns(df)

print(df_ver2.fillna(0))


           SREN        UBS       NOVN  SREN_return  UBS_return  NOVN_return
0     77.618309  15.735238  63.796165     0.000000    0.000000     0.000000
1     77.287575  15.735238  63.779427    -0.004261    0.000000    -0.000262
2     77.304970  15.848408  64.030525     0.000225    0.007192     0.003937
3     77.252754  15.975722  63.796165    -0.000675    0.008033    -0.003660
4     76.904610  15.697516  63.402779    -0.004507   -0.017414    -0.006166
..          ...        ...        ...          ...         ...          ...
586  109.000000  27.240000  88.110001     0.000000    0.041284     0.003759
587  109.849998  27.110001  88.800003     0.007798   -0.004772     0.007831
588  110.500000  27.700001  87.589996     0.005917    0.021763    -0.013626
589  111.800003  27.830000  87.129997     0.011765    0.004693    -0.005252
590  113.150002  27.809999  86.800003     0.012075   -0.000719    -0.003787

[591 rows x 6 columns]


In [5]:
#Covariance matrix
stock_returns = df_ver2[['SREN_return', 'UBS_return', 'NOVN_return']]
covariance = stock_returns.cov()
print(covariance)

             SREN_return  UBS_return  NOVN_return
SREN_return     0.000185    0.000139     0.000040
UBS_return      0.000139    0.000399     0.000044
NOVN_return     0.000040    0.000044     0.000121


In [6]:
# Array of stdev
stdv_SREN = round(np.sqrt(0.000185),5)
stdv_UBS = round(np.sqrt(0.000399),5)
stdv_NOVN = round(np.sqrt(0.000121),5)
port_smi = np.array([stdv_SREN,stdv_UBS,stdv_NOVN])
print(port_smi)


[0.0136  0.01997 0.011  ]


In [7]:
#Positions - adding new columns to a data frame
positions_of_shares = np.array([-50000,90000,65000])
def calculate_positions(df_ver2):
    positions_df_new = df_ver2.iloc[:, :3]*positions_of_shares
    positions_df_new.columns = [col + "_position" for col in df_ver2.columns[:3]]
    return pd.concat([df_ver2, positions_df_new], axis=1)
# data2 = {
#     'SREN': df_ver2['SREN'],
#     'UBS': df_ver2['UBS'],
#     'NOVN': df_ver2['NOVN']
# }
# df_ver2 = pd.DataFrame(data2)
df_ver_3 = calculate_positions(df_ver2)
df_ver_3.fillna(0)

,SREN,UBS,NOVN,SREN_return,UBS_return,NOVN_return,SREN_position,UBS_position,NOVN_position
0,77.618309,15.735238,63.796165,0.000000,0.000000,0.000000,-3880915.45,1416171.42,4146750.725
1,77.287575,15.735238,63.779427,-0.004261,0.000000,-0.000262,-3864378.75,1416171.42,4145662.755
2,77.304970,15.848408,64.030525,0.000225,0.007192,0.003937,-3865248.50,1426356.72,4161984.125
3,77.252754,15.975722,63.796165,-0.000675,0.008033,-0.003660,-3862637.70,1437814.98,4146750.725
4,76.904610,15.697516,63.402779,-0.004507,-0.017414,-0.006166,-3845230.50,1412776.44,4121180.635
...,...,...,...,...,...,...,...,...,...
586,109.000000,27.240000,88.110001,0.000000,0.041284,0.003759,-5450000.00,2451600.00,5727150.065
587,109.849998,27.110001,88.800003,0.007798,-0.004772,0.007831,-5492499.90,2439900.09,5772000.195
588,110.500000,27.700001,87.589996,0.005917,0.021763,-0.013626,-5525000.00,2493000.09,5693349.740
589,111.800003,27.830000,87.129997,0.011765,0.004693,-0.005252,-5590000.15,2504700.00,5663449.805


In [11]:
#Adding a column of total portfolio
def total_portfolio(df_ver3):
    total_position= df_ver3.iloc[:,6:9].sum(axis=1)
    total_position = total_position.to_frame(name="Total Portfolio")
    return pd.concat([df_ver3,total_position], axis=1)

df_ver_4 = total_portfolio(df_ver_3)
df_ver_4.fillna(0)

,SREN,UBS,NOVN,SREN_return,UBS_return,NOVN_return,SREN_position,UBS_position,NOVN_position,Total Portfolio
0,77.618309,15.735238,63.796165,0.000000,0.000000,0.000000,-3880915.45,1416171.42,4146750.725,1682006.695
1,77.287575,15.735238,63.779427,-0.004261,0.000000,-0.000262,-3864378.75,1416171.42,4145662.755,1697455.425
2,77.304970,15.848408,64.030525,0.000225,0.007192,0.003937,-3865248.50,1426356.72,4161984.125,1723092.345
3,77.252754,15.975722,63.796165,-0.000675,0.008033,-0.003660,-3862637.70,1437814.98,4146750.725,1721928.005
4,76.904610,15.697516,63.402779,-0.004507,-0.017414,-0.006166,-3845230.50,1412776.44,4121180.635,1688726.575
...,...,...,...,...,...,...,...,...,...,...
586,109.000000,27.240000,88.110001,0.000000,0.041284,0.003759,-5450000.00,2451600.00,5727150.065,2728750.065
587,109.849998,27.110001,88.800003,0.007798,-0.004772,0.007831,-5492499.90,2439900.09,5772000.195,2719400.385
588,110.500000,27.700001,87.589996,0.005917,0.021763,-0.013626,-5525000.00,2493000.09,5693349.740,2661349.830
589,111.800003,27.830000,87.129997,0.011765,0.004693,-0.005252,-5590000.15,2504700.00,5663449.805,2578149.655
